In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import random
#1

In [4]:
HY_model = tf.keras.models.load_model("Model/HY_Final_Model.h5")
FOG_model = tf.keras.models.load_model("Model/FOG_Final_Model.h5")
#2

## Physionet Dataset


In [5]:
features = ['Time', 'L1' , 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 
            'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 
            'Total_Force_Left', 'Total_Force_Right']

#3

In [ ]:
!mkdir CSV

# reading given csv file 
# and creating dataframe
for name in os.listdir('Physionet'):
  if 'Co' in name or 'Pt' in name:
    # print(name)
    df = pd.read_csv('Physionet/' + name, header = None, sep='\t')
      
    # adding column headings
    df.columns = features
      
    # store dataframe into csv file
    name = 'CSV/' + name.split('.')[0]+'.csv'
    # print(name)
    df.to_csv(name, index = None)

In [4]:
demographics = pd.read_csv('Physionet/demographics.txt', delim_whitespace=True)
sub_id = demographics.ID.to_list()
sub_names = []
for name in os.listdir('CSV'):
  sub_name = name.split('_')[0]
  sub_names.append(sub_name)

print("Subjects Count (in demographics) = ", len(sub_id))
print("Files count = ", len(sub_names))

Subjects Count (in demographics) =  166
Files count =  306


In [5]:
demographics.HoehnYahr
demographics['HoehnYahr'] = demographics['HoehnYahr'].fillna(0)
demographics.HoehnYahr.value_counts()

0.0    73
2.0    55
2.5    28
3.0    10
Name: HoehnYahr, dtype: int64

In [6]:
demographics['HoehnYahr'].replace({2.5: 1}, inplace=True)
demographics['HoehnYahr'] = demographics['HoehnYahr'].astype(int)
demographics.HoehnYahr.value_counts()

0    73
2    55
1    28
3    10
Name: HoehnYahr, dtype: int64

In [7]:
# !mkdir Final
# !mkdir Final/train
# !mkdir Final/test

count = len(os.listdir('CSV'))
train_count = int(70/100*count)
test_count = count - train_count

print("Training Count Subjects = ", train_count)
print("Test Count Subjects = ", test_count)

category = 'train'
counter = 1

ypath = 'Final/y_' + category + '.txt'
yfile = open(ypath, "a")
hypath = 'Final/hyscore_' + category + '.txt'
hyfile = open(hypath, "a")

for name in os.listdir('CSV'):
  if counter == train_count + 1:
    yfile.flush()
    yfile.close()
    hyfile.flush()
    hyfile.close()
    
    category = 'test'
    ypath = 'Final/y_' + category + '.txt'
    yfile = open(ypath, "a")
    hypath = 'Final/hyscore_' + category + '.txt'
    hyfile = open(hypath, "a")
  print(counter, name, category)
  
  sub_name = name.split('_')[0]
  sub_class = demographics[demographics['ID'] == sub_name]['Group'].to_string(index=False).strip()
  hy_class = demographics[demographics['ID'] == sub_name]['HoehnYahr'].to_string(index=False).strip()
  sub_data = pd.read_csv('CSV/' + name)
  features = sub_data.columns.to_list()

  full_size = 100
  overlap = 0.5
  overlap_size = int(full_size * overlap / 2)
  entry_size = full_size - overlap_size
  
  for i in range(0, sub_data.shape[0], entry_size):
    if sub_data.shape[0] >= i + entry_size + overlap_size:
      yfile.write(sub_class + "\n")
      hyfile.write(hy_class + "\n")
      
      for fname in features:
        path_name = 'Final/' + category + '/'
        file_name = fname + '_' + category + '.txt'
        with open(path_name + file_name, 'a') as feat_file:
          arr = sub_data.iloc[i:i + entry_size + overlap_size, ][fname].to_numpy()
          # print(i, len(arr), end = ' | ')
          feat_file.write(' '.join(map(str, arr)) + "\n")

  counter = counter + 1
yfile.flush()
yfile.close()
hyfile.flush()
yfile.close()

Training Count Subjects =  214
Test Count Subjects =  92
1 GaCo01_01.csv train
2 GaCo02_01.csv train
3 GaCo02_02.csv train
4 GaCo03_01.csv train
5 GaCo03_02.csv train
6 GaCo04_01.csv train
7 GaCo04_02.csv train
8 GaCo05_01.csv train
9 GaCo05_02.csv train
10 GaCo06_01.csv train
11 GaCo06_02.csv train
12 GaCo07_01.csv train
13 GaCo07_02.csv train
14 GaCo08_01.csv train
15 GaCo08_02.csv train
16 GaCo09_01.csv train
17 GaCo09_02.csv train
18 GaCo10_01.csv train
19 GaCo10_02.csv train
20 GaCo11_01.csv train
21 GaCo12_01.csv train
22 GaCo13_01.csv train
23 GaCo13_02.csv train
24 GaCo13_10.csv train
25 GaCo14_01.csv train
26 GaCo14_02.csv train
27 GaCo14_10.csv train
28 GaCo15_01.csv train
29 GaCo15_02.csv train
30 GaCo15_10.csv train
31 GaCo16_01.csv train
32 GaCo16_02.csv train
33 GaCo16_10.csv train
34 GaCo17_01.csv train
35 GaCo17_02.csv train
36 GaCo17_10.csv train
37 GaCo22_01.csv train
38 GaCo22_10.csv train
39 GaPt03_01.csv train
40 GaPt04_01.csv train
41 GaPt05_01.csv train
42 GaPt06

In [6]:
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files into a 3D array of [samples, timesteps, features]
def load_category(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded


# load a dataset category, such as train or test
def load_dataset_category(category, prefix=''):
	# load all 19 files as a single array
	filenames = []
	for fname in features:
		filenames.append(category + '/' + fname + '_' + category + '.txt')
  
	# load input data
	X = load_category(filenames, prefix)
	# load class output
	y = load_file(prefix + 'y_'+ category +'.txt')
	hy = load_file(prefix + 'hyscore_'+ category +'.txt')
	return X, y, hy

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy, trainHY = load_dataset_category('train', prefix)
	print(trainX.shape, trainy.shape, trainHY.shape)
	# load all test
	testX, testy, testHY = load_dataset_category('test', prefix)
	print(testX.shape, testy.shape, testHY.shape)
 
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = tf.keras.utils.to_categorical(trainy)
	trainHY = tf.keras.utils.to_categorical(trainHY)
	testy = tf.keras.utils.to_categorical(testy)
	testHY = tf.keras.utils.to_categorical(testHY)  
 
	# print(trainX.shape, trainy.shape, trainHY.shape, testX.shape, testy.shape, testHY.shape)
	return trainX, trainy, trainHY, testX, testy, testHY

#4

In [7]:
trainX, trainy, trainHY, testX, testy, testHY = load_dataset('Final/')
print("Training Data = ", trainX.shape)
print("Training Class = ", trainy.shape)
print("Training HoehnYahr Class = ", trainHY.shape)
print("Test Data = ", testX.shape)
print("Test Class = ", testy.shape)
print("Test HoehnYahr Class = ", testHY.shape)
#5

(30104, 100, 19) (30104, 1) (30104, 1)
(13761, 100, 19) (13761, 1) (13761, 1)
Training Data =  (30104, 100, 19)
Training Class =  (30104, 2)
Training HoehnYahr Class =  (30104, 4)
Test Data =  (13761, 100, 19)
Test Class =  (13761, 2)
Test HoehnYahr Class =  (13761, 3)


## Daphnet Dataset


In [8]:
class_var = 'annontations'
features = ['Time', 'ankle-x', 'ankle-y', 'ankle-z', 
            'thigh-x', 'thigh-y', 'thigh-z',
            'trunk-x', 'trunk-y', 'trunk-z', class_var]
len(os.listdir('dataset_fog_release/dataset'))
#6

17

In [ ]:
!mkdir FOG_CSV

# reading given csv file /
# and creating dataframe
for name in os.listdir('dataset_fog_release/dataset'):
    fog_df = pd.read_csv('dataset_fog_release/dataset/' + name, header = None, sep=' ')
      
    # adding column headings
    fog_df.columns = features
    fog_df = fog_df[fog_df.annontations != 0]  

    # store dataframe into csv file
    name = 'FOG_CSV/' + name.split('.')[0]+'.csv'
    # print(name)
    fog_df.to_csv(name, index = None)

In [11]:


count = len(os.listdir('CSV_D'))

train_count = int(85/100*count)
test_count = count - train_count

print("Training Count Subjects = ", train_count)
print("Test Count Subjects = ", test_count)

category = 'train'
counter = 1

ypath = 'Final_D/y_' + category + '.txt'
yfile = open(ypath, "a")

for name in sorted(os.listdir('CSV_D')):
  if counter == train_count + 1:
    yfile.flush()
    yfile.close()
    
    category = 'test'    
    ypath = 'Final_D/y_' + category + '.txt'
    yfile = open(ypath, "a")
  print(counter, name, category)
  
  sub_name = name.split('R')[0]
  sub_data = pd.read_csv('FOG_CSV/' + name)
  features = sub_data.columns.to_list()
  
  full_size = 128
  overlap = 0.5
  overlap_size = int(full_size * overlap / 2)
  entry_size = full_size - overlap_size

  for i in range(0, sub_data.shape[0], entry_size):
    if sub_data.shape[0] >= i + entry_size + overlap_size:
      sub_classes = sub_data.iloc[i:i+entry_size+overlap_size, ][class_var].to_list()
      sub_class = max(sub_classes, key = sub_classes.count)
      yfile.write(str(sub_class) + "\n")
 
      for fname in features:
        if fname != 'Time' and fname != class_var:
          path_name = 'Final_D/' + category + '/'
          file_name = fname + '_' + category + '.txt'
          with open(path_name + file_name, 'a') as feat_file:
            arr = sub_data.iloc[i:i+entry_size+overlap_size, ][fname].to_numpy()
            # print(i, len(arr), end = ' | ')
            feat_file.write(' '.join(map(str, arr)) + "\n")
  counter = counter + 1
yfile.flush()
yfile.close()

Training Count Subjects =  14
Test Count Subjects =  3
1 S01R01.csv train
2 S01R02.csv train
3 S02R01.csv train
4 S02R02.csv train
5 S03R01.csv train
6 S03R02.csv train
7 S03R03.csv train
8 S04R01.csv train
9 S05R01.csv train
10 S05R02.csv train
11 S06R01.csv train
12 S06R02.csv train
13 S07R01.csv train
14 S07R02.csv train
15 S08R01.csv test
16 S09R01.csv test
17 S10R01.csv test


In [9]:
def load_file_daphnet(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files into a 3D array of [samples, timesteps, features]
def load_category_daphnet(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file_daphnet(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded

# load a dataset category, such as train or test
def load_dataset_category_daphnet(category, prefix=''):
	# load all 10 files as a single array
  filenames = []
  for fname in features:
    if fname != 'Time' and fname != class_var:
      filenames.append(category + '/' + fname + '_' + category + '.txt')
  
  # load input data
  X = load_category_daphnet(filenames, prefix)
  # load class output
  y = load_file_daphnet(prefix + 'y_'+ category +'.txt')
  return X, y

# load the dataset, returns train and test X and y elements
def load_dataset_daphnet(prefix=''):
  # load all train
  trainX, trainy = load_dataset_category_daphnet('train', prefix)
  print(trainX.shape, trainy.shape)
  # load all test
  testX, testy = load_dataset_category_daphnet('test', prefix)
  print(testX.shape, testy.shape)

  # zero-offset class values
  trainy = trainy - 1
  testy = testy - 1

  neg, pos = np.bincount(trainy.reshape(1, trainy.shape[0])[0])
  total = neg + pos
  print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(total, pos, 100 * pos / total))
  # one hot encode y
  trainy = tf.keras.utils.to_categorical(trainy)
  testy = tf.keras.utils.to_categorical(testy)

  # print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  return trainX, trainy, testX, testy, neg, pos, total

#7

In [10]:
fog_trainX, fog_trainy, fog_testX, fog_testy, neg, pos, total = load_dataset_daphnet('Final_D/')
#8

(7932, 128, 9) (7932, 1)
(3158, 128, 9) (3158, 1)
Examples:
    Total: 7932
    Positive: 717 (9.04% of total)



In [14]:
print("Training Data = ", fog_trainX.shape)
print("Training Class = ", fog_trainy.shape)
print("Test Data = ", fog_testX.shape)
print("Test Class = ", fog_testy.shape)

Training Data =  (7932, 128, 9)
Training Class =  (7932, 2)
Test Data =  (3158, 128, 9)
Test Class =  (3158, 2)


## Final Prediction


In [11]:
def final_prediction(HY_model ,FOG_model, pd_data,fog_data, fog_class):
  # pd_data - 100 x 19
  # fog_data - 128 x 9
  hy_pred = np.argmax(HY_model.predict(np.expand_dims(pd_data, axis=0)))
  fog_pred = np.argmax(FOG_model.predict(np.expand_dims(fog_data, axis=0)))

  print("ORIGINAL CLASS")
  if np.argmax(fog_class) == 0:
      print("No Freeze")
  else:
    print("Freeze")

  print("-"*20)

  '''
    No Freeze - 0
    Freeze - 1
  '''
  
  print("PREDICTION")
  if fog_pred == 1:
    print("Freezing of GAIT")
  else:
    print("No Freezing of GAIT")


  
  '''
  Hoehn Yahr Scale
    0.0	- 0
    2.5	- 1
    2.0	- 2
    3.0	- 3
  '''
  print("Hoehn Yahr Scale Severity: ", end="")
  if hy_pred == 0:
    print("Healthy Control")
  elif hy_pred == 1:
    print("2.5 severity")
  elif hy_pred == 2:
    print("2 severity")
  elif hy_pred == 3:
    print("3 severity")
  
  print("_"*40, "\n")

#9

#Random Prediction

In [38]:
for x in range(5):
  i = random.randrange(0, min(len(testX), len(fog_testX)))
  final_prediction(HY_model,FOG_model, testX[i], fog_testX[i], fog_testy[i])
  #10

ORIGINAL CLASS
No Freeze
--------------------
PREDICTION
No Freezing of GAIT
Hoehn Yahr Scale Severity: 2 severity
________________________________________ 

ORIGINAL CLASS
No Freeze
--------------------
PREDICTION
No Freezing of GAIT
Hoehn Yahr Scale Severity: 2 severity
________________________________________ 

ORIGINAL CLASS
No Freeze
--------------------
PREDICTION
No Freezing of GAIT
Hoehn Yahr Scale Severity: Healthy Control
________________________________________ 

ORIGINAL CLASS
No Freeze
--------------------
PREDICTION
No Freezing of GAIT
Hoehn Yahr Scale Severity: 2.5 severity
________________________________________ 

ORIGINAL CLASS
No Freeze
--------------------
PREDICTION
No Freezing of GAIT
Hoehn Yahr Scale Severity: 2 severity
________________________________________ 

ORIGINAL CLASS
No Freeze
--------------------
PREDICTION
No Freezing of GAIT
Hoehn Yahr Scale Severity: 2.5 severity
________________________________________ 

ORIGINAL CLASS
No Freeze
------------------